In [67]:
from langchain_openai import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler

llm = ChatOpenAI(
    temperature=0.1,
    model="gpt-4o-mini",
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

In [68]:
from langchain.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate, MessagesPlaceholder
from langchain.prompts.pipeline import PipelinePromptTemplate

examples = [
    {
        "question": "Moana",
        "answer": """
        🌊👩💪
        """,
    },
    {
        "question": "The Godfather",
        "answer": """
        👨‍👨‍👦🔫🍝
        """,
    },
    {
        "question": "Top Gun",
        "answer": """
        🛩️👨‍✈️🔥
        """,
    },
    {
        "question": "Home Alone",
        "answer": """
        👦🏠🎄
        """,
    },
    {
        "question": "The Lion King",
        "answer": """
        🦁👑🌅
        """,
    },
]

example_template = ChatPromptTemplate.from_template(
    """
    This is an example of how you talk:
    human :{question},
    You : {answer},
    """
)

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_template,
    examples=examples,
)

history_prompt = ChatPromptTemplate.from_template(
    """
    This is an history between you and human:
    {history}
    """
)

start_prompt = ChatPromptTemplate.from_template(
    """
    Start now!
    DON'T ever confuse what was in the example with history.
    When Human ask about the conversation record, You answer in English by referring to the conversation record, and when Human ask about the movie title, YouI answer with an emoji by referring to the example.
    When you use emojis, you have to use ONLY three.
    Human: {question}
    You:
    """
)

prompts = [("example", example_prompt), ("history", history_prompt), ("start", start_prompt)]

final_prompt = ChatPromptTemplate.from_template(
    """
    {example}
    Example Done!
    ----------
    {history}
    History Done!
    ----------
    {start}
    """
)

full_prompt = PipelinePromptTemplate(
    final_prompt=final_prompt,
    pipeline_prompts=prompts
)

In [69]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(
    llm=llm,
    max_token_limit=500,
    memory_key="history",
)


def memory_save(input, output):
    memory.save_context({"input": input}, {"output" :  output})


def memory_load(_):
    return memory.load_memory_variables({})["history"]

In [70]:
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda

chain = (
    {"history": RunnableLambda(memory_load), "question": RunnablePassthrough(),}
    | full_prompt
    | llm
)

In [71]:
def invoke_chain(question):
    result = chain.invoke(question).content
    memory_save(question, result)

In [72]:
invoke_chain("Titanic")
invoke_chain("The Avengers")

🚢💔🌊🦸‍♂️🦸‍♀️🌍

In [73]:
chain.invoke("In reference to conversation records, what was the first movie I asked?").content

The first movie you asked about was "Titanic."

'The first movie you asked about was "Titanic."'